In [1]:
import warnings
warnings.filterwarnings("ignore")
import wrangle as w

import pandas as pd
import numpy as np

#splits
from sklearn.model_selection import train_test_split

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

#scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, classification_report

#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
import kc_wrangle as w
import wrangle as ww


# Acquire

In [2]:
df = pd.read_csv("merged_data - merged_data.csv")

In [3]:
df, model_df = w.get_prep_aa(df)

In [9]:
def split_data(df, target_variable):
    '''
    Takes in two arguments the dataframe name and the ("target_variable" - must be in string format) to stratify  and 
    return train, validate, test subset dataframes will output train, validate, and test in that order.
    '''
    train, test = train_test_split(df, #first split
                                   test_size=.2, 
                                   random_state=123, 
                                   stratify= df[target_variable])
    train, validate = train_test_split(train, #second split
                                    test_size=.25, 
                                    random_state=123, 
                                    stratify=train[target_variable])
    return train, validate, test

In [11]:
train, validate, test = split_data(df, "outcome")

In [12]:
train

,has_name,outcome,dob,species,intake_type,intake_condition,intake_date,outcome_date,intake_age,outcome_age,tenure_days,intake_sex,breed,mix_breeds,two_breeds,pure_breed,primary_color,is_tabby,mix_color
159865,1,transfer,2013-08-22,dog,stray,normal,2015-08-22,2015-10-06,730.0,775,45.0,spayed female,chihuahua shorthair mix,1,0,0,tan,0,0
111780,1,adoption,2012-06-18,dog,owner surrender,sick,2019-06-21,2019-07-21,2559.0,2589,30.0,intact male,great pyrenees mix,1,0,0,white,0,0
109945,1,return to owner,2018-02-18,dog,stray,normal,2019-02-18,2019-02-19,365.0,366,1.0,intact male,irish terrier,0,0,0,black,0,1
157772,0,transfer,2011-04-08,cat,stray,injured,2016-04-07,2016-04-09,1826.0,1828,2.0,neutered male,domestic shorthair mix,1,0,0,orange,1,0
178967,1,adoption,2018-01-21,dog,stray,normal,2019-01-21,2019-02-04,365.0,379,14.0,intact male,plott hound mix,1,0,0,white,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125693,0,transfer,2016-04-30,cat,stray,normal,2017-04-30,2017-05-10,365.0,375,10.0,intact male,domestic shorthair mix,1,0,0,black,0,0
159681,1,adoption,2013-04-28,dog,owner surrender,normal,2014-04-28,2014-05-03,365.0,370,5.0,spayed female,gordon setter mix,1,0,0,black,0,1
74894,0,transfer,2021-05-05,dog,stray,normal,2023-05-05,2023-05-11,730.0,736,6.0,intact female,black mouth cur mix,1,0,0,tan,0,1
40508,0,adoption,2014-06-21,dog,stray,normal,2014-08-06,2014-08-20,46.0,60,14.0,intact male,beagle mix,1,0,0,tricolor,0,1


* lowercase variables
* intake age/ outcome age leave just days
* name column made need to be dropped from too many nulls 
* animal id column change to id 


In [ ]:
df.head()

# Prepare

In [ ]:
#find numeric columns
num_col = df.select_dtypes(include='number').columns
num_col

In [ ]:
#get string columns
str_col = ['outcome', 'intake_type', 'intake_condition',
       'species', 'found_location', 'intake_sex', 'breed', 'color']

In [ ]:
df.intake_sex.value_counts()

In [ ]:
df.intake_condition.value_counts()

rename intake_conditions: Normal, Injured, Immediate attn needed, mental, delayed/terminal

In [ ]:
df.head()

In [ ]:
# lowercase everything
df = df.apply(lambda x: x.astype(str).str.lower())

# top ten colors and other for erybody else, or for simplify all colors

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion